## 필요한 패키지 임포트 및 메소드 생성

In [182]:
## 패키지 설치
# pip install google-api-python-client

In [1]:
import pandas as pd
# anaconda환경 : conda install -c conda-forge google-api-python-client
from googleapiclient.discovery import build

def read_api_key():
    infile = open('C:/Users/4ser/Desktop/이도원/기타잡것/key_data/youtube_api_key_1.txt', 'r')
    api_key = infile.readline()
    return api_key

In [184]:
video_id = 'tZolEtoQyuY'
comments = list()
api_obj = build('youtube', 'v3', developerKey=read_api_key())

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/4ser/Desktop/이도원/기타잡것/key_data/youtube_api_key_1.txt'

## response

- response 1회당 1건씩 API 할당량 소모. while문은 maxResults 수치에 따라 다름. 예시) maxResults=100 : 할당량 약 4회
- part='snippet,replies'에서 snippet은 json파일의 최상단 key값이라 생각하면 됨. (필수조건)
- maxResults=10000은 최대로 가져올 데이터의 갯수를 10000개로 제한.
- 이렇게 가져온 response는 'items'라는 key값을 가지며, `response['items']`는 JSON파일로 나타냄.

In [ ]:
# https://www.youtube.com/watch?part='snippet,replies'&vieoid='asdasd'&maxResults=100이와 같이 요청을 함 snippet필수속성, replies댓글
response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=1000).execute() 

In [ ]:
while response:
    for item in response['items']:        
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
        if item['snippet']['totalReplyCount'] > 0:
            for reply_item in item['replies']['comments']:
                reply = reply_item['snippet']
                comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
 
    if 'nextPageToken' in response:
        response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
    else:
        break

In [ ]:
response['items']

[{'kind': 'youtube#commentThread',
  'etag': 'cac_RZOGXNWPSAlOe_0JP0Be5Qk',
  'id': 'Ugwm6rBc-G9J6j2tCRx4AaABAg',
  'snippet': {'videoId': 'tZolEtoQyuY',
   'topLevelComment': {'kind': 'youtube#comment',
    'etag': '4xcAztyLhqHNQfGOQgvvH6KXa88',
    'id': 'Ugwm6rBc-G9J6j2tCRx4AaABAg',
    'snippet': {'videoId': 'tZolEtoQyuY',
     'textDisplay': '인트로 진짜 아기 시무룩했다가 다시 좋아하는거같네ㅋㅋㅋㅋㅋㅋ 왜이리커엽냐',
     'textOriginal': '인트로 진짜 아기 시무룩했다가 다시 좋아하는거같네ㅋㅋㅋㅋㅋㅋ 왜이리커엽냐',
     'authorDisplayName': '트오밥',
     'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AL5GRJUC18-1DuTjvb8N-WmaM6LC-1iKONdQF4FwHw=s48-c-k-c0x00ffffff-no-rj',
     'authorChannelUrl': 'http://www.youtube.com/channel/UCrNbLJUDU5BE8tTU_JFL8gg',
     'authorChannelId': {'value': 'UCrNbLJUDU5BE8tTU_JFL8gg'},
     'canRate': True,
     'viewerRating': 'none',
     'likeCount': 9,
     'publishedAt': '2022-12-04T10:23:27Z',
     'updatedAt': '2022-12-04T10:23:27Z'}},
   'canReply': True,
   'totalReplyCount': 0,
   'isPublic': True}},
 {'ki

## 내보내기

### 데이터프레임화 및 csv로 내보내기

In [ ]:
df = pd.DataFrame(comments)
df.columns = ['comment', 'author', 'date', 'num_likes']
# df.to_csv('results.xlsx', index=None)
df

,comment,author,date,num_likes
0,"<a href=""https://www.youtube.com/watch?v=tZolE...",박민수,2023-02-12T13:12:25Z,0
1,"<a href=""https://www.youtube.com/watch?v=tZolE...",우름,2023-02-03T17:30:47Z,0
2,라꼰 시즌4 얼릉 해줘여 ㅠ,ㅎㅎㅎ,2023-02-03T02:19:01Z,0
3,먹방 방송중 자기들이 먹기 전에 스테프들 먼저 챙겨 주는 먹방은 처음보네요<br>침...,youngjin na,2023-02-02T21:27:53Z,1
4,아 진짜 만화가들의 저 상상력과 입담...쵝오야..끝까지 순수함을 지켜죠 ㅠ..,Punk-in Pie ASMR,2023-02-01T03:33:36Z,0
...,...,...,...,...
778,과연 몇이나 될까…,개삭수네,2022-12-14T09:05:53Z,0
779,장난치고 안먹으면 쳐맞아야 되지만<br>다먹음 ㅋㅋㅋㅋㅋㅋㅋ,Ne Beul,2022-12-07T08:33:24Z,1
780,하지만 맛있었죠?,두부,2022-12-05T14:57:31Z,2
781,탈모팸 1쉬,ㄷㅎㄱㅈㅇ,2022-12-05T05:47:38Z,0


In [ ]:
df.sort_values(by='num_likes', ascending=False)

,comment,author,date,num_likes
776,&quot;음식 가지고 장난치는 아재팸 1위&quot;,GeeChan Lee,2022-12-04T10:01:00Z,3323
568,다 커서도 이런 친구를 만날수 있다는게 넘 부럽다,크로커스,2022-12-04T10:51:57Z,1073
711,아 오늘 일요일이여서 기분 별로 였는데 저녁 보면서 보는 침착맨 개꿀잼이다...,물먹는미역,2022-12-04T10:05:22Z,1051
380,침펄이 풍한테 형형거리면서 노는거 왜케 귀엽냐,Cool태수,2022-12-04T13:41:27Z,1022
625,"<a href=""https://www.youtube.com/watch?v=tZolE...",강한별,2022-12-04T10:31:06Z,950
...,...,...,...,...
280,생신축하드려요,규민,2022-12-04T16:59:23Z,0
279,"<a href=""https://www.youtube.com/watch?v=tZolE...",XXX MADunknown,2022-12-04T17:02:44Z,0
278,"<a href=""https://www.youtube.com/watch?v=tZolE...",ㅡ,2022-12-04T17:09:13Z,0
277,칙촉이랑 제크 사오래요 왜케 얄밉냐ㅋㅋ,와플,2022-12-04T17:12:08Z,0


## JSON 파일로 내보내기

In [ ]:
import json

# response['items'] 변수
items = response['items']

# 저장할 파일 경로
file_path = './items.json'

# json 파일로 내보내기
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(items, json_file)

In [ ]:
import Preprocessing as prep

In [ ]:
import pandas as pd

# pd.DataFrame

prep.apply_regular_expression("그타6 언제 나오냐%^ 락스타 ㅊㅌ개새끼들!")

# prep.remove_one_letter_noun

AttributeError: module 'Preprocessing' has no attribute 'apply_regular_expression'

## 캐글 데이터셋

Mounted at /content/drive


In [1]:
## 파일 경로
## 문장
# file_path_train_hate = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/train.hate.csv"
# file_path_dev_hate = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/dev.hate.csv"
# file_path_unlabeled_comments = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/unlabeled_comments.txt"
# file_path_no_hate = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/test.hate.no_label.csv"
# file_path_kold_v1 = "/content/drive/MyDrive/공부/자연어 처리/자료/kold_v1.json"
# file_path_youtube_comments = "/content/drive/MyDrive/공부/자연어 처리/자료/comments_update_df.csv"
file_path_trip = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/tripadviser_review.csv"
# file_path_drama = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/Dataset.csv"
file_path_common = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/common_word.csv"

# ## 단어
file_path_pos_word = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/pos_word.csv"
file_path_neg_word = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/neg_word.csv"
# file_path_unknown = "/content/drive/MyDrive/공부/자연어 처리/자료/KnuSentiLex/obj_unknown_pol_word.txt"
# file_path_senti_dict = "/content/drive/MyDrive/공부/자연어 처리/자료/KnuSentiLex/SentiWord_Dict.txt"
file_path_stopwords = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/stopwords.txt"

In [2]:
"""
[문장]
df_train_hate : 
 - label 있음.
 - none         3486
 - offensive    2499
 - hate         1911


df_dev_hate :
 - label 있음.
 - offensive    189
 - none         160
 - hate         122


df_unlabeled_comments : label 없음
df_no_hate : label 없음
df_kold_v1 : label 없음
df_youtube_comments : label 없음,  좋아요 수 있음


[단어]
pos_word.csv
neg_word.csv
obj_unknown_pol_word.txt
SentiWord_Dict.txt
"""

import pandas as pd


## 문장
# df_train_hate = pd.read_csv(file_path_train_hate, encoding='utf-8', engine='python')
# df_dev_hate = pd.read_csv(file_path_dev_hate, encoding='utf-8', engine='python')
# df_unlabeled_comments = pd.read_table(file_path_unlabeled_comments, encoding='utf-8', engine='python', names=['comment'],)
# df_no_hate = pd.read_csv(file_path_no_hate, encoding='utf-8', engine='python')
# df_kold_v1 = pd.read_json(file_path_kold_v1, encoding='utf-8')
# df_youtube_comments = pd.read_csv(file_path_youtube_comments, encoding='utf-8', engine='python')
df_trip = pd.read_csv(file_path_trip, encoding='utf-8', engine='python')
df_kdrama_common = pd.read_csv(file_path_common, encoding='utf-8', engine='python',)


## 단어
df_pos_word = pd.read_csv(file_path_pos_word, encoding='utf-8', engine='python')
df_neg_word = pd.read_csv(file_path_neg_word, encoding='utf-8', engine='python')
# df_unknown_word = pd.read_table(file_path_unknown, encoding='utf-8', engine='python', names=['word'])

In [3]:
def invert_rating(rating):
    y_value = 0
    if rating >= 4:
        y_value = 2
    elif rating == 3:
        y_value = 1
    elif rating <= 2:
        y_value = 0
    return y_value

In [4]:
import pandas as pd
import numpy as np


df_trip['y'] = df_trip.rating.apply(invert_rating)

df_pos_word.label = df_pos_word.word
df_pos_word.word = np.nan
df_pos_word.columns = ['rating', 'text', 'y']

df_neg_word.label = df_neg_word.word
df_neg_word.word = np.nan
df_neg_word.columns = ['rating', 'text', 'y']

new_df = pd.concat([df_trip, df_pos_word, df_neg_word], axis=0)
new_df = pd.concat([new_df, df_kdrama_common], axis=0)
new_df.drop(['rating'], axis=1, inplace=True)

new_df_m1 = new_df.loc[new_df.y == -1].sample(5300, random_state=33)
new_df_copy = new_df.loc[(new_df.y == 0) | (new_df.y == 1), :]
new_df = pd.concat([new_df_m1, new_df_copy], axis=0)
new_df.reset_index(drop=True, inplace=True)
new_df

,text,y
0,억지로 참거나,-1
1,변태,-1
2,받지 못하게,-1
3,맞갖지 않게,-1
4,몹시 아쉽다,-1
...,...,...
15433,호랑이형 12년전 녹양동작업실앞에서 사진도찍어주시고 사인해주신거 정말감사했습니다.!,0
15434,어쩐대~~~김 정훈 진짜 진아 조아하던것가뜨만,0
15435,캬 큐브 먹여살린 현아를 퇴출이라니.. 초대형 FA나왔구나 모셔갈라고 경쟁 장난 아닐듯,0
15436,이거 보다 더한 막장의 드라마도 넋놓고 보면서 무슨?,0


In [5]:
file_path = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/changed_label.csv"
new_df = pd.read_csv(file_path, encoding='utf-8', engine='python')
new_df

,text,y
0,여행에 집중할수 있게 편안한 휴식을 제공하는 호텔이었습니다. 위치선정 또한 적당한 ...,3
1,"2일 이상 연박시 침대, 이불, 베게등 침구류 교체 및 어메니티 보강이 필요해 보입...",3
2,지인에소개로온 호텔 깨끗하고 좋은거같아요 처음에는 없는게 많아 많이 당황했는데 ...,3
3,방에 딱 들어서자마자 눈이 휘둥그레질정도로 이렇게 넓은 호텔 처음 와본 것 같아요!...,3
4,저녁에 맥주한잔 하는게 좋아서 렌트 안하고 뚜벅이 하기로 했는데 호텔 바로 앞에 버...,3
...,...,...
16159,안정감,3
16160,적극적이다,3
16161,승리,3
16162,승리하다,3


In [12]:
new_df.y.value_counts()

3    5572
1    5420
2    5172
Name: y, dtype: int64

In [13]:
new_df.isnull().sum()

text    0
y       0
dtype: int64

## 자연어 처리 시작

### 자연어 처리 모듈 임포트

In [6]:
import Preprocessing as prep
import Modeling as mo

In [7]:
stopwords_list = ['제주도', '호스텔', '위치', '대중교통', '타고', '어디', '공항', '잠깐', '직원', '택시', '상품',]
stopwords_list = prep.stopwords(file_path_stopwords, stopwords_list)

In [8]:
prep.text_cleaning(new_df.text[5400], stopwords_list)

['어쩌면', '이다해', '승리', '승리']

### 문서-단어 행렬 생성

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(tokenizer= lambda x: prep.text_cleaning(x, stopwords_list))
bow_vect = vect.fit_transform(new_df['text'].tolist())

In [44]:
word_list = vect.get_feature_names_out()
count_list = bow_vect.toarray().sum(axis=0)
count_list

array([ 5,  1, 15, ...,  1,  1,  1], dtype=int64)

In [45]:
word_count_dict = dict(zip(word_list, count_list))
word_count_dict

{'가가': 5,
 '가가린': 1,
 '가게': 15,
 '가격': 262,
 '가격표': 1,
 '가관': 2,
 '가구': 12,
 '가급': 1,
 '가기': 23,
 '가까이': 24,
 '가끔': 10,
 '가나': 4,
 '가난': 5,
 '가난뱅이': 1,
 '가년': 1,
 '가능': 12,
 '가능성': 4,
 '가다가': 1,
 '가담': 1,
 '가당': 1,
 '가도': 8,
 '가동': 2,
 '가두': 1,
 '가드': 1,
 '가득': 5,
 '가든': 2,
 '가라': 6,
 '가락': 2,
 '가래': 2,
 '가량': 5,
 '가려움': 1,
 '가렴': 1,
 '가렵': 1,
 '가로': 7,
 '가로본능폰': 1,
 '가로세로': 3,
 '가로수길': 1,
 '가루': 2,
 '가르침': 3,
 '가마니': 1,
 '가면': 21,
 '가몬': 1,
 '가무': 1,
 '가문': 1,
 '가물': 1,
 '가뭄': 2,
 '가미': 1,
 '가방': 4,
 '가버': 1,
 '가보': 1,
 '가봉': 1,
 '가쁘': 8,
 '가쁜': 2,
 '가사': 5,
 '가성': 51,
 '가세': 9,
 '가소': 1,
 '가수': 29,
 '가스': 1,
 '가스레인지': 1,
 '가스렌지': 1,
 '가슴': 22,
 '가슴앓이': 2,
 '가시': 4,
 '가식': 2,
 '가신': 4,
 '가안': 1,
 '가야': 20,
 '가오': 1,
 '가옥': 1,
 '가요': 17,
 '가요무대': 1,
 '가요제': 1,
 '가용': 2,
 '가운데': 4,
 '가운데손가락': 1,
 '가위': 1,
 '가을': 5,
 '가인': 3,
 '가장': 55,
 '가점': 1,
 '가정': 14,
 '가정불화': 1,
 '가정사': 2,
 '가정식': 2,
 '가정폭력': 3,
 '가족': 137,
 '가족사': 1,
 '가족사진': 1,
 '가족영화': 2,
 '가즈': 11,
 '가지': 77,
 '가짐': 3,
 '가짓수':

### TF-IDF 생성

In [46]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
tfidf_vect = tfidf.fit_transform(bow_vect)
tfidf_vect.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### 단어 맵핑

In [47]:
invert_index_vectorizer = {v : k for k, v in vect.vocabulary_.items()}
invert_index_vectorizer[2928]

'무궁무진'

In [48]:
vect.vocabulary_

{'여행': 5870,
 '집중': 8197,
 '휴식': 10086,
 '제공': 7680,
 '호텔': 9883,
 '선정': 4441,
 '청소': 8419,
 '청결': 8413,
 '상태': 4285,
 '침대': 8673,
 '이불': 6794,
 '침구': 8672,
 '교체': 821,
 '어메니티': 5652,
 '보강': 3608,
 '베스트': 3539,
 '웨스턴': 6447,
 '회원': 10025,
 '경우': 487,
 '객실': 299,
 '층수': 8635,
 '요청': 6274,
 '적극': 7441,
 '반영': 3288,
 '지인': 8087,
 '소개': 4594,
 '처음': 8367,
 '당황': 1838,
 '세면': 4546,
 '도구': 1994,
 '잠옷': 7290,
 '필수': 9439,
 '그것': 1000,
 '정도': 7594,
 '박만': 3221,
 '저녁': 7422,
 '맥주': 2691,
 '한잔': 9578,
 '렌트': 2382,
 '뚜벅': 2290,
 '버스': 3468,
 '정류': 7599,
 '도착': 2046,
 '가방': 47,
 '일찍': 7087,
 '정비': 7614,
 '방이': 3382,
 '입실': 7136,
 '업그레이드': 5747,
 '정말': 7604,
 '바다': 3183,
 '전망': 7484,
 '전일': 7516,
 '함덕': 9610,
 '대명': 1871,
 '콘도': 8799,
 '실내': 5198,
 '분위기': 3843,
 '손님': 4679,
 '가장': 79,
 '배치': 3428,
 '대해': 1933,
 '대응': 1898,
 '써비스': 5292,
 '조식': 7770,
 '부분': 3763,
 '신경': 5119,
 '아주': 5376,
 '특급': 9086,
 '트랜디': 9059,
 '고민': 557,
 '흔적': 10116,
 '여름': 5838,
 '수영장': 4788,
 '사용': 4142,
 '보고': 3612,
 '엄마':

## 모델링

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

In [20]:
x_train, x_test, y_train, y_test = train_test_split(tfidf_vect, new_df['y'], shuffle=True, test_size=0.2, stratify=new_df.y)

model = RandomForestClassifier(random_state=33)
y_pred = model.fit(x_train, y_train).predict(x_test)


In [21]:
accuracy_score(y_pred, y_test)

0.702752861119703

In [21]:
None, 'micro', 'macro', 'weighted', 'samples'

(None, 'micro', 'macro', 'weighted', 'samples')

In [22]:
recall_score(y_pred, y_test, average='macro')

0.7578452601597299

In [23]:
precision_score(y_pred, y_test, average='macro')

0.6977772561464155

In [24]:
f1_score(y_pred, y_test, average='macro')

0.691854518971323

In [25]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           1       0.91      0.55      0.69      1787
           2       0.75      0.92      0.83       846
           3       0.43      0.80      0.56       600

    accuracy                           0.69      3233
   macro avg       0.70      0.76      0.69      3233
weighted avg       0.78      0.69      0.70      3233



In [26]:
confusion_matrix(y_pred, y_test)

array([[986, 211, 590],
       [ 26, 777,  43],
       [ 72,  46, 482]], dtype=int64)

In [17]:
import Preprocessing as prep

prep.invert_text_to_vect(new_df.loc[0, 'text'])

FileNotFoundError: [Errno 2] No such file or directory: ''

In [30]:
new_df.y.value_counts()

-1    5300
 0    5137
 1    5001
Name: y, dtype: int64

## ㄹㅇㄴㅁㄹㅇㄴㅁ


In [3]:
!pip list | grep konlpy

konlpy                        0.5.2


In [ ]:
!pip install konlpy==0.5.2 jpype1 Jpype1-py3

In [36]:
none_df['oriented_index'] = none_df.index
none_df.reset_index(drop=True, inplace=True)
none_df

,comments,label,oriented_index
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,none,0
1,아이즈원 힘내세요...일본 진출도 했으니 일본서 좋은 모습 보여줘도 팬들은 응원 합니다.,none,6
2,강부자 선생님 전미선 비보에 오열을 하셨다니 눈물이 나네요 힘내세요,none,7
3,이경규가 이런거보면 세련되긴함 저 나이에 차은우 누가 알꼬 아무리 잘생겼다해도. 배...,none,10
4,진짜 멋지네요~,none,12
...,...,...,...
3641,힘내세요~ 응원합니다!!,none,7891
3642,힘내세요~~삼가 고인의 명복을 빕니다..,none,7892
3643,힘내세용 ^^ 항상 응원합니닷 ^^ !,none,7893
3644,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,none,7894


In [54]:
train_none_hate = pd.read_csv('train_none_hate.csv', encoding='utf-8', engine='python')
train_none_hate

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,negative
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",negative
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,negative
...,...,...
7891,힘내세요~ 응원합니다!!,none
7892,힘내세요~~삼가 고인의 명복을 빕니다..,none
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,none
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,none


In [55]:
train_none_hate.label.value_counts()

negative    4410
none        3486
Name: label, dtype: int64

In [59]:
df_kold_v1

,guid,source,date,title,comment,OFF,TGT,GRP,OFF_span,TGT_span,raw_labels
0,kold-v1_00000,naver_news,2021-07-25,페미니즘이 범죄가 되는 나라 [삶과 문화],남녀평등 주장할 거면 여성징병제에도 동의하라고ㅋㅋㅋ 그리고 내 말에 그냥 시비만 걸...,True,group,others-feminist,쳐받으면서 왜 차별받는다고 말하는지 말해보라고ㅋㅋㅋ,,"[{'offensiveness': True, 'annotator_id': 19151..."
1,kold-v1_00001,naver_news,2021-04-13,[젠더의학①] 여성은 ‘몸집 작은 남성’이 아니다,의학에도 젠더 사상이 붙네 ㄷㄷ;,True,untargeted,None,젠더 사상이 붙네 ㄷㄷ;,,"[{'offensiveness': False, 'annotator_id': 1856..."
2,kold-v1_00002,naver_news,2021-09-15,“책 생명 늘려야죠”… 문학 속 ‘성차별’ 패치 떼는 출판계,"루브르 박물관에 있는 모나리자 머리도 단발로 수정하고,미국에 있는 자유의 여신상도 ...",True,group,others-feminist,"모나리자 머리도 단발로 수정하고,미국에 있는 자유의 여신상도 콘크리트 부어서 가슴부...",,"[{'offensiveness': True, 'annotator_id': 18177..."
3,kold-v1_00003,naver_news,2021-01-14,이루다로 촉발된 젠더 논쟁... 개인정보 유출이란 본질 외면,진짜 어이가 없네 딥페이크 만든 사람들도 처벌하고 알페스 만든 사람들도 처벌하라니깐...,True,untargeted,None,왜 쫄리나..?,,"[{'offensiveness': True, 'annotator_id': 16616..."
4,kold-v1_00004,naver_news,2021-05-06,"GS25, 브레이브걸스 포스터 또 젠더 이슈 논란",브레이브걸스=페미아이돌,True,group,others-others,페미아이돌,브레이브걸스,"[{'offensiveness': True, 'annotator_id': 18808..."
...,...,...,...,...,...,...,...,...,...,...,...
40424,kold-v1_40424,naver_news,2021-07-29,"""미국 국무부, 中의 탈북 난민 보호 이행 촉구""",우리나라 민주당은 탈북민들을 배신자라고 표현하지.,True,group,politics-progressive,탈북민들을 배신자라고 표현하지.,민주당,"[{'offensiveness': True, 'annotator_id': 16218..."
40425,kold-v1_40425,naver_news,2020-11-14,조계종 종평위원장 도심 스님 “종교 혐오 막으려면 차별금지법부터”,차별금지법..반대..,True,untargeted,None,,,"[{'offensiveness': True, 'annotator_id': 23603..."
40426,kold-v1_40426,naver_news,2021-02-01,"""짜증나는 조선족"" vs ""우리 중국동포"" 때아닌 호칭 논란, 어떻게 생각하십니까",다른 나라 사람을 동포라 부르고 중국 조선족을 동포라 안부르니까 문제가 되는거다 동...,False,None,None,,,"[{'offensiveness': False, 'annotator_id': 5913..."
40427,kold-v1_40427,youtube,2021-08-17,"제1회 차별금지법 바로알기 아카데미, 김지연 교수",지금은 죽도록 하나님의 진리를 지켜 충성된 주의 백성임을 입증해야할 때입니다.,False,None,None,,,"[{'offensiveness': False, 'annotator_id': 1566..."


In [ ]:
df_kold_v1.loc[:, ['comment', ]]